In [29]:
import pandas as pd

train = pd.read_csv('./open/train.csv')
test = pd.read_csv('./open/test.csv')


data_onehot_enco = pd.read_csv('./open/onehot_encoded_시,일제외.csv', encoding = "UTF_8", low_memory=False)
data_onehot_enco = data_onehot_enco.drop(['Unnamed: 0'], axis=1)


In [30]:
# train_new가 기존 train
# test_new가 기존 test
train_enco = data_onehot_enco[:len(train)]
test_enco = data_onehot_enco[len(train):]
# 동제거
train_drop_dong = [col for col in train_enco.columns if col.startswith("동_")]
train_filtered = train_enco.drop(columns=train_drop_dong, inplace=False)

test_drop_dong = [col for col in test_enco.columns if col.startswith("동_")]
test_filtered = test_enco.drop(columns=test_drop_dong, inplace=False)


In [32]:
y_train_new = train['ECLO'].copy()


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_filtered, y_train_new, test_size=0.3, random_state=42)

In [34]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# 선형 회귀 모델 생성 및 훈련
model_Lir = LinearRegression()
model_Lir.fit(X_train, y_train)

# 검증 데이터에 대한 예측 수행
y_pred = model_Lir.predict(X_valid)

# Mean Squared Error (MSE) 값 계산
mse = mean_squared_error(y_valid, y_pred)
print(f'Mean Squared Error (MSE) 값: {mse:.4f}')

# R-squared (R2) 값 계산
r2 = r2_score(y_valid, y_pred)
print(f'R-squared (R2) 값: {r2:.4f}')

# Root Mean Squared Logarithmic Error (RMSLE) 값 계산
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

rmsle_value = rmsle(y_valid, y_pred)
print(f'RMSLE 값: {rmsle_value:.4f}')

Mean Squared Error (MSE) 값: 9.7529
R-squared (R2) 값: 0.0238
RMSLE 값: 0.4652


# 가중치 확인

In [35]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27726 entries, 29542 to 15795
Data columns (total 85 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   요일_금요일           27726 non-null  float64
 1   요일_목요일           27726 non-null  float64
 2   요일_수요일           27726 non-null  float64
 3   요일_월요일           27726 non-null  float64
 4   요일_일요일           27726 non-null  float64
 5   요일_토요일           27726 non-null  float64
 6   요일_화요일           27726 non-null  float64
 7   기상상태_기타          27726 non-null  float64
 8   기상상태_눈           27726 non-null  float64
 9   기상상태_맑음          27726 non-null  float64
 10  기상상태_비           27726 non-null  float64
 11  기상상태_안개          27726 non-null  float64
 12  기상상태_흐림          27726 non-null  float64
 13  노면상태_건조          27726 non-null  float64
 14  노면상태_기타          27726 non-null  float64
 15  노면상태_서리/결빙       27726 non-null  float64
 16  노면상태_적설          27726 non-null  float64
 17  노면상태_젖음/습기   

In [8]:
# 특성별 계수 확인
coefficients = pd.Series(model_Lir.coef_, index=X_train.columns)
coefficients = coefficients.abs().sort_values(ascending=False)
# 초기 가중치
initial_coefficients = pd.Series(model_Lir.coef_, index=X_train.columns)
# 가장 큰 영향을 끼치는 특성 확인
most_influential_feature = coefficients.index[0]
print("초기 가중치:")

# 출력 옵션 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# 모든 가중치 출력
# print(initial_coefficients)
# 원래의 출력 옵션으로 돌아가기
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

print(f'가장 큰 영향을 끼치는 특성: {most_influential_feature}')

초기 가중치:
가장 큰 영향을 끼치는 특성: 연_2022


# 가중치 조절은 선형회귀 부적합. 랜포로

In [9]:
from sklearn.ensemble import RandomForestRegressor

# 랜덤 포레스트 모델 생성 및 훈련
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)

# 검증 데이터에 대한 예측 수행
y_pred_rf = model_rf.predict(X_valid)

# Root Mean Squared Logarithmic Error (RMSLE) 값 계산
rmsle_value_rf = rmsle(y_valid, y_pred_rf)
print(f'RMSLE 값 (랜덤 포레스트): {rmsle_value_rf:.4f}')

RMSLE 값 (랜덤 포레스트): 0.4952


In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27726 entries, 29542 to 15795
Data columns (total 85 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   요일_금요일           27726 non-null  float64
 1   요일_목요일           27726 non-null  float64
 2   요일_수요일           27726 non-null  float64
 3   요일_월요일           27726 non-null  float64
 4   요일_일요일           27726 non-null  float64
 5   요일_토요일           27726 non-null  float64
 6   요일_화요일           27726 non-null  float64
 7   기상상태_기타          27726 non-null  float64
 8   기상상태_눈           27726 non-null  float64
 9   기상상태_맑음          27726 non-null  float64
 10  기상상태_비           27726 non-null  float64
 11  기상상태_안개          27726 non-null  float64
 12  기상상태_흐림          27726 non-null  float64
 13  노면상태_건조          27726 non-null  float64
 14  노면상태_기타          27726 non-null  float64
 15  노면상태_서리/결빙       27726 non-null  float64
 16  노면상태_적설          27726 non-null  float64
 17  노면상태_젖음/습기   

In [11]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# 랜덤 포레스트 모델 생성 및 훈련
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)

# 특성 중요도 확인
feature_importances = model_rf.feature_importances_

# 특성 중요도에 기반한 가중치 설정
top_feature_indices = np.argsort(feature_importances)[::-1][:3]
weight_factor = 4  # 가중치 조절을 위한 상수

# 가중치 적용
for idx in top_feature_indices:
    feature_importances[idx] *= weight_factor

# 가중치가 적용된 특성의 이름 추출
weighted_feature_names = X_train.columns[top_feature_indices]

print("가중치가 적용된 특성의 이름:")
for feature_name in weighted_feature_names:
    print(feature_name)
# 가중치 적용한 모델 재생성
model_rf_weighted = RandomForestRegressor()
model_rf_weighted.fit(X_train, y_train)

# 가중치 적용한 모델로 예측 수행
y_pred_rf_weighted = model_rf_weighted.predict(X_valid)

# RMSLE 계산
rmsle_value_rf_weighted = rmsle(y_valid, y_pred_rf_weighted)
print(f'RMSLE 값 (가중치 적용): {rmsle_value_rf_weighted:.4f}')



가중치가 적용된 특성의 이름:
구_수성구
요일_금요일
월_6
RMSLE 값 (가중치 적용): 0.4945


가중치 1.5일때
RMSLE 값 (가중치 적용): 0.4883

In [15]:
train_sub = model_Lir.predict(test_enco)

In [14]:
#기존 제출파일 불러오기 
sub = pd.read_csv('./open/sample_submission.csv') 

In [ ]:
# 제출용 파일 만들기
sub['ECLO'] = train_sub
submission_df = sub[['ID', 'ECLO']]
# CSV 파일로 저장
submission_df.to_csv('Linear_Regression_submission.csv', index=False)